### combine all features of docuentation of web scrapping
step1 : driver main url of housing.com

step 2 : i will get list of area in form of list

step 3: search for the particular area in housing.com

step4 : scrape data from each card by using switch tabs and closing them 

step 5 : once desired number of data is scraped we are going to return to home page and repeat the process



In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import pandas as pd
chrome_options = Options()
chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
driver.get("https://housing.com/")

In [51]:
chennai_localities = [
    "Adyar, Chennai",
    "Anna Nagar, Chennai",
    "T. Nagar, Chennai",
    "Velachery, Chennai",
    "Tambaram, Chennai",
    "Porur, Chennai",
    "Sholinganallur, Chennai",
    "Guindy, Chennai",
    "Kodambakkam, Chennai",
    "Thiruvanmiyur, Chennai",
    "Nungambakkam, Chennai",
    "Mylapore, Chennai",
    "Perambur, Chennai",
    "Saidapet, Chennai",
    "Pallavaram, Chennai",
    "Ambattur, Chennai",
    "Kilpauk, Chennai",
    "Royapettah, Chennai",
    "Egmore, Chennai",
    "Besant Nagar, Chennai",
    "Ashok Nagar, Chennai",
    "Medavakkam, Chennai",
    "Chromepet, Chennai",
    "Manapakkam, Chennai",
    "Madipakkam, Chennai"
]

### searching each and every locality to get data 
WE CAN USE BACK METHOD TO GET BACK TO MAIN PAGE

In [ ]:
data_of_all_property = []

def get_text(xpath):
    try:
        return driver.find_element(By.XPATH, xpath).text
    except:
        return "N/A"

def safe_print(label, xpath):
    try:
        property_data[label] = get_text(xpath)  # ← store in dict
    except Exception as e:
        property_data[label] = "N/A"

def get_by_label_th(label):
    try:
        xpath = f"//tr[th[contains(text(),'{label}')]]/td/div"
        return driver.find_element(By.XPATH, xpath).text
    except:
        return "N/A"

for location in chennai_localities:
    search_bar = driver.find_element("xpath",'/html/body/div[1]/div[1]/div[4]/div/div/div[1]/div/div[2]/input')
    search_bar.send_keys(location)
    time.sleep(2)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(3)
    property_filter = driver.find_element("xpath","/html/body/div[1]/div[1]/div[3]/div[1]/div/div[1]")
    property_filter.click()
    try:
        independent_house = driver.find_element("xpath","/html/body/div[1]/div[1]/div[3]/div[1]/div/div[1]/div[2]/div/div/div/div[2]/span/a")
        independent_house.click()
    except :
        print("failed")
    # driver url is updated now like https://housing.com/in/buy/chennai/kodungaiyur_west
    # we can start fetch data from property cards 
    for no_of_card in range(1,11):
        # entering into url of seprate property
        time.sleep(4)
        property_page = driver.current_window_handle
        try:
            property_link = driver.find_element("xpath",f"/html/body/div[1]/div[1]/div[3]/div[2]/div[1]/div[4]/div[{no_of_card}]/div/article/div[1]/div[2]/div[1]/div[1]/div[1]/a")
        except:
            continue
        time.sleep(3)
        property_link.click()

        # keeping track of all tab to ensure smooth transmission
        all_tabs_browser = driver.window_handles
        #selecting tab that is not main one
        property_details_tab = [tab for tab in all_tabs_browser if tab!=property_page][0]
        driver.switch_to.window(property_details_tab)

        #### ---------------------------------------- actually fetch part ------------------------------------------
        property_data={}
        property_data["Title"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/h1/div[1]/div")
        property_data["Price"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/div[2]/div/div[2]/div[1]/span")
        property_data["Area"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[1]/div[1]/div/div")
        property_data["Avg Price"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[2]/div[1]")
        property_data["Facing"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[4]/div[1]")
        property_data["Furnishing"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[5]/div[1]")

        property_data["Bedrooms"] = get_by_label_th("Bedrooms")
        property_data["Bathrooms"] = get_by_label_th("Bathrooms")
        property_data["Balcony"] = get_by_label_th("Balcony")

        # Handle score containers
        containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'T_featureContainerStyle')]")
        for container in containers:
            try:
                label = container.find_element(By.TAG_NAME, "span").text
                score = container.find_element(By.TAG_NAME, "p").text
                property_data[label] = score
            except:
                continue
        #append the new data
        data_of_all_property.append(property_data)
        driver.close()
        driver.switch_to.window(property_page)
    #for bringing the property page to home again
    driver.back()
    driver.back()
driver.quit()


WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=137.0.7151.56)
Stacktrace:
	GetHandleVerifier [0x0x11ef173+62931]
	GetHandleVerifier [0x0x11ef1b4+62996]
	(No symbol) [0x0x1000ece]
	(No symbol) [0x0xff1870]
	(No symbol) [0x0xff09e1]
	(No symbol) [0x0x100e7ef]
	(No symbol) [0x0x107515c]
	(No symbol) [0x0x108f2f9]
	(No symbol) [0x0x106e2a6]
	(No symbol) [0x0x103d5f0]
	(No symbol) [0x0x103e464]
	GetHandleVerifier [0x0x1443463+2504899]
	GetHandleVerifier [0x0x143e892+2485490]
	GetHandleVerifier [0x0x121590a+220522]
	GetHandleVerifier [0x0x1206388+157672]
	GetHandleVerifier [0x0x120cb6d+184269]
	GetHandleVerifier [0x0x11f7158+95672]
	GetHandleVerifier [0x0x11f7300+96096]
	GetHandleVerifier [0x0x11e21aa+9738]
	BaseThreadInitThunk [0x0x76bb5d49+25]
	RtlInitializeExceptionChain [0x0x77ddd09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x77ddd021+561]


## convert it into csv file 



In [ ]:
## convert it into csv file 

df = pd.DataFrame(data_of_all_property)
df.to_csv("propert_data.csv",index=False)
print("data stored sucessfully")

data stored sucessfully
